# Analisi Back Squat con Accelerometro

Questo notebook analizza i dati dell'accelerometro durante l'esecuzione di back squat, identificando automaticamente le diverse fasi del movimento:
- **Posizione iniziale (Standing)**
- **Fase eccentrica (Discesa)**
- **Posizione inferiore (Bottom)**
- **Fase concentrica (Salita)**

I dati vengono visualizzati con marker che indicano i cambiamenti di fase.

## 1. Import Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter1d

# Configurazione per grafici più grandi e leggibili
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerie importate con successo")

In [1]:
%pip install pandas scipy

Note: you may need to restart the kernel to use updated packages.


## 2. Caricamento Dati

In [ ]:
# Carica il file CSV
filename = 'accel_data_20251118_074755.csv'
df = pd.read_csv(filename)

# Mostra le prime righe
print(f"📊 Dataset caricato: {len(df)} campioni")
print(f"⏱️  Durata: {df['Timestamp'].max() - df['Timestamp'].min():.2f} secondi")
print(f"📈 Frequenza di campionamento: ~{len(df)/(df['Timestamp'].max() - df['Timestamp'].min()):.1f} Hz\n")

df.head(10)

## 3. Visualizzazione Dati Grezzi

In [ ]:
# Plot dei dati grezzi
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

axes[0].plot(df['Timestamp'], df['X (g)'], 'r-', alpha=0.7, linewidth=1)
axes[0].set_ylabel('X (g)', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Accelerazione sui 3 assi + Magnitudine', fontsize=14, fontweight='bold')

axes[1].plot(df['Timestamp'], df['Y (g)'], 'g-', alpha=0.7, linewidth=1)
axes[1].set_ylabel('Y (g)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df['Timestamp'], df['Z (g)'], 'b-', alpha=0.7, linewidth=1)
axes[2].set_ylabel('Z (g)', fontweight='bold')
axes[2].grid(True, alpha=0.3)

axes[3].plot(df['Timestamp'], df['Magnitude (g)'], 'purple', linewidth=2)
axes[3].set_ylabel('Magnitudine (g)', fontweight='bold')
axes[3].set_xlabel('Tempo (s)', fontweight='bold')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Smoothing e Preprocessing

In [ ]:
# Applica smoothing gaussiano per ridurre il rumore
sigma = 2  # Parametro di smoothing

df['X_smooth'] = gaussian_filter1d(df['X (g)'], sigma=sigma)
df['Y_smooth'] = gaussian_filter1d(df['Y (g)'], sigma=sigma)
df['Z_smooth'] = gaussian_filter1d(df['Z (g)'], sigma=sigma)
df['Mag_smooth'] = gaussian_filter1d(df['Magnitude (g)'], sigma=sigma)

# Calcola la derivata (velocità) della magnitudine per identificare cambiamenti
df['Mag_velocity'] = np.gradient(df['Mag_smooth'], df['Timestamp'])
df['Mag_velocity_smooth'] = gaussian_filter1d(df['Mag_velocity'], sigma=sigma)

print("✅ Smoothing applicato")

## 5. Rilevamento Fasi dello Squat

L'algoritmo identifica l'**inizio** di ogni fase del back squat:
1. **Baseline** - Posizione iniziale in piedi (stabile)
2. **Eccentrica** - Inizio discesa (accelerazione negativa su Y)
3. **Concentrica** - Inizio spinta verso l'alto (punto più basso + cambio direzione)
4. **Arresto** - Fine movimento (decelerazione vicino al top)
5. **Baseline** - Ritorno alla stabilità

In [ ]:
# Identificazione delle 5 fasi del back squat - SINGOLA RIPETIZIONE
# Analisi basata su stati: Baseline → Eccentrica → Bottom → Concentrica → Baseline

mag = df['Mag_smooth'].values
y_acc = df['Y_smooth'].values
timestamps = df['Timestamp'].values

# Definisci soglie per identificare gli stati
baseline_threshold = 0.05  # Soglia per considerare baseline stabile (vicino a 1g)

print("🔍 ANALISI STATI - SINGOLA RIPETIZIONE BACK SQUAT")
print("=" * 80)

# STATO 1: Baseline iniziale - stabilità prima del movimento
# Cerca la prima zona stabile vicino a 1g
baseline_stable = np.where(np.abs(mag - 1.0) < baseline_threshold)[0]
if len(baseline_stable) > 10:
    baseline_1_idx = baseline_stable[0]
else:
    baseline_1_idx = 0

print(f"✅ STATO 1 - Baseline iniziale: t={timestamps[baseline_1_idx]:.2f}s (Mag={mag[baseline_1_idx]:.3f}g)")

# STATO 2: Eccentrica - prima rottura della baseline (discesa)
# Cerca il primo punto dove la magnitudine aumenta significativamente dalla baseline
eccentric_candidates = np.where((mag > mag[baseline_1_idx] + 0.1) & 
                                (np.arange(len(mag)) > baseline_1_idx))[0]
if len(eccentric_candidates) > 0:
    eccentric_start_idx = eccentric_candidates[0]
else:
    # Fallback: cerca quando mag supera 1.1g
    eccentric_start_idx = np.where(mag > 1.1)[0][0] if len(np.where(mag > 1.1)[0]) > 0 else baseline_1_idx + 10

print(f"✅ STATO 2 - Eccentrica (discesa): t={timestamps[eccentric_start_idx]:.2f}s (Mag={mag[eccentric_start_idx]:.3f}g)")

# STATO 3: Bottom - punto più basso (minimo locale della magnitudine dopo l'eccentrica)
# Cerca il minimo globale dopo l'inizio dell'eccentrica
search_window = mag[eccentric_start_idx:]
if len(search_window) > 0:
    bottom_relative_idx = np.argmin(search_window)
    bottom_idx = eccentric_start_idx + bottom_relative_idx
else:
    bottom_idx = eccentric_start_idx + 20

print(f"✅ STATO 3 - Bottom (posizione più bassa): t={timestamps[bottom_idx]:.2f}s (Mag={mag[bottom_idx]:.3f}g)")

# STATO 4: Concentrica - spinta verso l'alto (grande accelerazione)
# Cerca il picco massimo di magnitudine dopo il bottom
search_window_concentric = mag[bottom_idx:]
if len(search_window_concentric) > 0:
    concentric_peak_relative_idx = np.argmax(search_window_concentric)
    concentric_peak_idx = bottom_idx + concentric_peak_relative_idx
else:
    concentric_peak_idx = bottom_idx + 10

print(f"✅ STATO 4 - Concentrica (spinta/picco accelerazione): t={timestamps[concentric_peak_idx]:.2f}s (Mag={mag[concentric_peak_idx]:.3f}g)")

# STATO 5: Baseline finale - ritorno alla stabilità dopo la concentrica
# Cerca quando torna vicino a 1g dopo il picco concentrico
stable_after_concentric = np.where((np.abs(mag[concentric_peak_idx:] - 1.0) < baseline_threshold))[0]
if len(stable_after_concentric) > 5:
    baseline_2_idx = concentric_peak_idx + stable_after_concentric[5]  # Prendi qualche campione dopo per essere sicuro
else:
    baseline_2_idx = min(concentric_peak_idx + 20, len(mag) - 1)

print(f"✅ STATO 5 - Baseline finale (ritorno a stabilità): t={timestamps[baseline_2_idx]:.2f}s (Mag={mag[baseline_2_idx]:.3f}g)")

# Crea struttura phases con una singola ripetizione
phases = [{
    'rep': 1,
    'baseline_1': baseline_1_idx,
    'eccentric': eccentric_start_idx,
    'concentric': bottom_idx,  # Il bottom è l'inizio della fase concentrica
    'arrest': concentric_peak_idx,  # Il picco è l'arresto
    'baseline_2': baseline_2_idx
}]

print("\n" + "=" * 80)
print(f"📊 SINGOLA RIPETIZIONE IDENTIFICATA")
print(f"   Durata totale: {timestamps[baseline_2_idx] - timestamps[baseline_1_idx]:.2f}s")
print(f"   Durata fase eccentrica: {timestamps[bottom_idx] - timestamps[eccentric_start_idx]:.2f}s")
print(f"   Durata fase concentrica: {timestamps[concentric_peak_idx] - timestamps[bottom_idx]:.2f}s")
print(f"   Magnitudine minima (bottom): {mag[bottom_idx]:.3f}g")
print(f"   Magnitudine massima (concentrica): {mag[concentric_peak_idx]:.3f}g")
print(f"   Range di accelerazione: {mag[concentric_peak_idx] - mag[bottom_idx]:.3f}g")
print("=" * 80)

## 6. Visualizzazione con Marker delle Fasi

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Magnitudine con marker delle fasi
ax1 = axes[0]
ax1.plot(df['Timestamp'], df['Magnitude (g)'], 'lightgray', alpha=0.5, linewidth=1, label='Raw')
ax1.plot(df['Timestamp'], df['Mag_smooth'], 'purple', linewidth=2, label='Smoothed')

# Marker per tutte le fasi
colors = {
    'baseline_1': ('blue', 's', 'Baseline Start'),
    'eccentric': ('red', 'v', 'Eccentrica (Discesa)'),
    'concentric': ('green', '^', 'Concentrica (Spinta)'),
    'arrest': ('orange', 'D', 'Arresto'),
    'baseline_2': ('cyan', 's', 'Baseline End')
}

for phase in phases:
    for phase_name, (color, marker, label) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            # Plot marker solo per la prima ripetizione per evitare duplicati nella legenda
            if phase['rep'] == 1:
                ax1.plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Mag_smooth'], 
                        marker=marker, color=color, markersize=12, label=label, zorder=5)
            else:
                ax1.plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Mag_smooth'], 
                        marker=marker, color=color, markersize=12, zorder=5)

ax1.set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=12)
ax1.set_title('Analisi Fasi Back Squat - Marker all\'Inizio di Ogni Fase', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right', fontsize=9, ncol=2)

# Plot 2: Asse Y (verticale) con marker
ax2 = axes[1]
ax2.plot(df['Timestamp'], df['Y (g)'], 'lightgreen', alpha=0.4, linewidth=1, label='Raw')
ax2.plot(df['Timestamp'], df['Y_smooth'], 'darkgreen', linewidth=2, label='Smoothed')

# Marker per tutte le fasi sull'asse Y
for phase in phases:
    for phase_name, (color, marker, label) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            if phase['rep'] == 1:
                ax2.plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Y_smooth'], 
                        marker=marker, color=color, markersize=12, label=label, zorder=5)
            else:
                ax2.plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Y_smooth'], 
                        marker=marker, color=color, markersize=12, zorder=5)

ax2.set_xlabel('Tempo (s)', fontweight='bold', fontsize=12)
ax2.set_ylabel('Y - Asse Verticale (g)', fontweight='bold', fontsize=12)
ax2.set_title('Accelerazione Verticale (Y) con Fasi', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(loc='upper right', fontsize=9, ncol=2)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)

plt.tight_layout()
plt.show()

## 7. Grafico Multi-Asse Completo

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(16, 12), sharex=True)

# Definizione marker
colors = {
    'baseline_1': ('blue', 's', 'Baseline Start'),
    'eccentric': ('red', 'v', 'Eccentrica'),
    'concentric': ('green', '^', 'Concentrica'),
    'arrest': ('orange', 'D', 'Arresto'),
    'baseline_2': ('cyan', 's', 'Baseline End')
}

# X axis
axes[0].plot(df['Timestamp'], df['X_smooth'], 'r-', linewidth=2)
for phase in phases:
    for phase_name, (color, marker, _) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            axes[0].plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['X_smooth'], 
                        marker=marker, color=color, markersize=8)
axes[0].set_ylabel('X (g)', fontweight='bold', fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Analisi Completa Back Squat - Marker all\'Inizio delle Fasi', fontsize=14, fontweight='bold')

# Y axis (verticale)
axes[1].plot(df['Timestamp'], df['Y_smooth'], 'g-', linewidth=2)
for phase in phases:
    for phase_name, (color, marker, label) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            if phase['rep'] == 1:  # Legenda solo per prima ripetizione
                axes[1].plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Y_smooth'], 
                            marker=marker, color=color, markersize=8, label=label)
            else:
                axes[1].plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Y_smooth'], 
                            marker=marker, color=color, markersize=8)
axes[1].set_ylabel('Y (g)', fontweight='bold', fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='upper right', fontsize=9, ncol=3)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)

# Z axis
axes[2].plot(df['Timestamp'], df['Z_smooth'], 'b-', linewidth=2)
for phase in phases:
    for phase_name, (color, marker, _) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            axes[2].plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Z_smooth'], 
                        marker=marker, color=color, markersize=8)
axes[2].set_ylabel('Z (g)', fontweight='bold', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Magnitude
axes[3].plot(df['Timestamp'], df['Mag_smooth'], 'purple', linewidth=2.5)
for phase in phases:
    for phase_name, (color, marker, _) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            axes[3].plot(df.iloc[idx]['Timestamp'], df.iloc[idx]['Mag_smooth'], 
                        marker=marker, color=color, markersize=10, zorder=5)
            # Linea verticale per separare le ripetizioni
            if phase_name == 'baseline_1' and phase['rep'] > 1:
                axes[3].axvline(x=df.iloc[idx]['Timestamp'], color='gray', 
                               linestyle=':', alpha=0.5, linewidth=1)

axes[3].set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=11)
axes[3].set_xlabel('Tempo (s)', fontweight='bold', fontsize=11)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Analisi Statistica delle Ripetizioni

In [ ]:
# Analizza ogni ripetizione identificata
if len(phases) > 0:
    print(f"\n📊 ANALISI DETTAGLIATA RIPETIZIONI SQUAT")
    print("=" * 100)
    
    squat_data = []
    
    for phase in phases:
        rep_num = phase['rep']
        
        # Timestamp di ogni fase
        t_baseline1 = df.iloc[phase['baseline_1']]['Timestamp']
        t_eccentric = df.iloc[phase['eccentric']]['Timestamp']
        t_concentric = df.iloc[phase['concentric']]['Timestamp']
        t_arrest = df.iloc[phase['arrest']]['Timestamp']
        t_baseline2 = df.iloc[phase['baseline_2']]['Timestamp']
        
        # Calcola durate delle fasi
        eccentric_duration = t_concentric - t_eccentric  # Tempo di discesa
        concentric_duration = t_arrest - t_concentric     # Tempo di spinta
        total_rep_time = t_baseline2 - t_baseline1        # Tempo totale ripetizione
        
        # Valori di accelerazione
        mag_baseline1 = df.iloc[phase['baseline_1']]['Mag_smooth']
        mag_eccentric = df.iloc[phase['eccentric']]['Mag_smooth']
        mag_concentric = df.iloc[phase['concentric']]['Mag_smooth']
        mag_arrest = df.iloc[phase['arrest']]['Mag_smooth']
        mag_baseline2 = df.iloc[phase['baseline_2']]['Mag_smooth']
        
        squat_data.append({
            'Rep': rep_num,
            'Baseline Start (s)': f"{t_baseline1:.2f}",
            'Eccentrica Start (s)': f"{t_eccentric:.2f}",
            'Concentrica Start (s)': f"{t_concentric:.2f}",
            'Arresto (s)': f"{t_arrest:.2f}",
            'Baseline End (s)': f"{t_baseline2:.2f}",
            'Durata Eccentrica (s)': f"{eccentric_duration:.2f}",
            'Durata Concentrica (s)': f"{concentric_duration:.2f}",
            'Tempo Totale (s)': f"{total_rep_time:.2f}",
            'Mag Bottom (g)': f"{mag_concentric:.3f}",
            'Mag Peak (g)': f"{mag_arrest:.3f}"
        })
    
    # Crea DataFrame per visualizzazione
    squat_df = pd.DataFrame(squat_data)
    print(squat_df.to_string(index=False))
    print("=" * 100)
    
    # Statistiche generali
    print(f"\n📈 STATISTICHE GENERALI")
    print(f"   Numero ripetizioni: {len(phases)}")
    
    eccentric_times = [float(d['Durata Eccentrica (s)']) for d in squat_data]
    concentric_times = [float(d['Durata Concentrica (s)']) for d in squat_data]
    
    print(f"   Durata media fase eccentrica: {np.mean(eccentric_times):.2f}s (±{np.std(eccentric_times):.2f}s)")
    print(f"   Durata media fase concentrica: {np.mean(concentric_times):.2f}s (±{np.std(concentric_times):.2f}s)")
    print(f"   Ratio eccentrica/concentrica: {np.mean(eccentric_times)/np.mean(concentric_times):.2f}")
    
else:
    print("⚠️ Nessuna ripetizione completa rilevata nel dataset")

## 8b. Calcolo Mean Velocity (VBT - Velocity Based Training)

In [ ]:
# Calcolo della Mean Velocity per ogni ripetizione
# Nel VBT, la velocità media è calcolata integrando l'accelerazione verticale (asse Y)
# durante la fase concentrica (spinta verso l'alto)

print(f"\n🚀 VELOCITY BASED TRAINING (VBT) - MEAN VELOCITY")
print("=" * 100)

vbt_data = []

for phase in phases:
    rep_num = phase['rep']
    
    # Estrai la fase concentrica (dalla posizione più bassa fino all'arresto)
    start_idx = phase['concentric']
    end_idx = phase['arrest']
    
    # Dati della fase concentrica
    time_concentric = timestamps[start_idx:end_idx+1]
    y_accel_concentric = y_acc[start_idx:end_idx+1]
    
    # Rimuovi la gravità (assumendo che l'asse Y è verticale e 1g = gravità)
    # In posizione statica, Y dovrebbe essere ~0g se il sensore è orientato correttamente
    # L'accelerazione netta è quella che causa il movimento
    y_accel_net = y_accel_concentric * 9.81  # Converti da g a m/s²
    
    # Integra l'accelerazione per ottenere la velocità
    # Usa integrazione trapezoidale
    dt = np.diff(time_concentric)
    velocity = np.zeros(len(y_accel_net))
    
    for i in range(1, len(velocity)):
        # Integrazione trapezoidale: v[i] = v[i-1] + (a[i-1] + a[i])/2 * dt
        velocity[i] = velocity[i-1] + (y_accel_net[i-1] + y_accel_net[i]) / 2 * dt[i-1]
    
    # Mean Velocity: media della velocità durante la fase concentrica
    mean_velocity = np.mean(np.abs(velocity))
    
    # Peak Velocity: massima velocità raggiunta
    peak_velocity = np.max(np.abs(velocity))
    
    # Durata fase concentrica
    concentric_duration = time_concentric[-1] - time_concentric[0]
    
    # Displacement (spostamento) stimato: integrale della velocità
    displacement = np.trapz(np.abs(velocity), time_concentric)
    
    vbt_data.append({
        'Rep': rep_num,
        'Mean Velocity (m/s)': f"{mean_velocity:.3f}",
        'Peak Velocity (m/s)': f"{peak_velocity:.3f}",
        'Durata Concentrica (s)': f"{concentric_duration:.2f}",
        'Displacement (m)': f"{displacement:.3f}",
        'Power Indicator': f"{mean_velocity / concentric_duration:.2f}" if concentric_duration > 0 else "N/A"
    })

# Crea DataFrame VBT
vbt_df = pd.DataFrame(vbt_data)
print(vbt_df.to_string(index=False))
print("=" * 100)

# Statistiche VBT
mean_velocities = [float(d['Mean Velocity (m/s)']) for d in vbt_data]
peak_velocities = [float(d['Peak Velocity (m/s)']) for d in vbt_data]

print(f"\n📊 STATISTICHE VBT")
print(f"   Mean Velocity media: {np.mean(mean_velocities):.3f} m/s (±{np.std(mean_velocities):.3f} m/s)")
print(f"   Peak Velocity media: {np.mean(peak_velocities):.3f} m/s (±{np.std(peak_velocities):.3f} m/s)")
print(f"\n💡 NOTE:")
print(f"   - Velocità calcolata dalla fase concentrica (spinta verso l'alto)")
print(f"   - Mean Velocity è la metrica principale del VBT")
print(f"   - Valori tipici back squat: 0.3-0.6 m/s (carico alto), 0.6-1.0 m/s (carico medio)")
print(f"   - Power Indicator = Mean Velocity / Tempo (indica esplosività)")

# Visualizza profilo velocità per ogni ripetizione
fig, axes = plt.subplots(len(phases), 1, figsize=(14, 4*len(phases)), sharex=False)
if len(phases) == 1:
    axes = [axes]

for idx, phase in enumerate(phases):
    rep_num = phase['rep']
    start_idx = phase['concentric']
    end_idx = phase['arrest']
    
    time_concentric = timestamps[start_idx:end_idx+1]
    y_accel_concentric = y_acc[start_idx:end_idx+1]
    y_accel_net = y_accel_concentric * 9.81
    
    # Ricalcola velocità per il plot
    dt = np.diff(time_concentric)
    velocity = np.zeros(len(y_accel_net))
    for i in range(1, len(velocity)):
        velocity[i] = velocity[i-1] + (y_accel_net[i-1] + y_accel_net[i]) / 2 * dt[i-1]
    
    # Plot
    ax = axes[idx]
    ax.plot(time_concentric, velocity, 'b-', linewidth=2, label='Velocity')
    ax.axhline(y=float(vbt_data[idx]['Mean Velocity (m/s)']), color='red', 
               linestyle='--', linewidth=2, label=f"Mean Velocity = {vbt_data[idx]['Mean Velocity (m/s)']} m/s")
    ax.fill_between(time_concentric, 0, velocity, alpha=0.3, color='blue')
    
    ax.set_ylabel('Velocity (m/s)', fontweight='bold', fontsize=11)
    ax.set_xlabel('Time (s)', fontweight='bold', fontsize=11)
    ax.set_title(f'Rep {rep_num} - Velocity Profile (Fase Concentrica)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n✅ Analisi VBT completata per {len(phases)} ripetizioni")

## 9. Visualizzazione 3D della Traiettoria

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot della traiettoria 3D
ax.plot(df['X_smooth'], df['Y_smooth'], df['Z_smooth'], 
        'gray', linewidth=1.5, alpha=0.4, label='Traiettoria')

# Marker per ogni fase
colors = {
    'baseline_1': ('blue', 's', 100, 'Baseline Start'),
    'eccentric': ('red', 'v', 150, 'Eccentrica'),
    'concentric': ('green', '^', 150, 'Concentrica'),
    'arrest': ('orange', 'D', 120, 'Arresto'),
    'baseline_2': ('cyan', 's', 100, 'Baseline End')
}

for phase in phases:
    for phase_name, (color, marker, size, label) in colors.items():
        if phase_name in phase:
            idx = phase[phase_name]
            if phase['rep'] == 1:  # Legenda solo per prima ripetizione
                ax.scatter(df.iloc[idx]['X_smooth'], 
                          df.iloc[idx]['Y_smooth'], 
                          df.iloc[idx]['Z_smooth'],
                          c=color, s=size, marker=marker, edgecolors='black', linewidths=1.5,
                          label=label, zorder=5)
            else:
                ax.scatter(df.iloc[idx]['X_smooth'], 
                          df.iloc[idx]['Y_smooth'], 
                          df.iloc[idx]['Z_smooth'],
                          c=color, s=size, marker=marker, edgecolors='black', linewidths=1.5,
                          zorder=5)

ax.set_xlabel('X (g)', fontweight='bold', fontsize=11)
ax.set_ylabel('Y (g)', fontweight='bold', fontsize=11)
ax.set_zlabel('Z (g)', fontweight='bold', fontsize=11)
ax.set_title('Traiettoria 3D - Marker all\'Inizio di Ogni Fase', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3)

# Imposta vista ottimale
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

## 10. Riepilogo e Conclusioni

Questo notebook ha analizzato i dati dell'accelerometro durante l'esecuzione di back squat identificando le **5 fasi** del movimento:

### Fasi Identificate:
1. 🟦 **Baseline Start** - Posizione iniziale stabile in piedi
2. 🔴 **Eccentrica** - Inizio della discesa (fase negativa)
3. 🟢 **Concentrica** - Inizio della spinta verso l'alto (punto più basso)
4. 🟠 **Arresto** - Decelerazione e fine del movimento verso l'alto
5. 🔵 **Baseline End** - Ritorno alla posizione stabile

### Marker sui Grafici:
- **Quadrati blu** (□) = Baseline
- **Triangoli rossi giù** (▼) = Inizio fase eccentrica
- **Triangoli verdi su** (▲) = Inizio fase concentrica
- **Diamanti arancioni** (◆) = Arresto
- **Quadrati ciano** (□) = Ritorno baseline

### Analisi Completata:
✅ Identificazione automatica delle transizioni di fase  
✅ Marker visualizzati all'**inizio** di ogni fase  
✅ Analisi temporale delle durate eccentriche/concentriche  
✅ Statistiche complete per ogni ripetizione  
✅ Visualizzazione 3D della traiettoria completa